In [8]:
import numpy as np
import pandas as pd
from sklearn.datasets import make_blobs
from sklearn.metrics.pairwise import cosine_similarity

In [9]:
class MyDBSCAN:
    def __init__(self, eps=3, min_samples=3, metric='euclidian'):
        # Инициализация параметров eps и min_samples
        self.eps = eps
        self.min_samples = min_samples
        self.metric = metric
        
    def __str__(self):
        # Формируем строку с параметрами экземпляра
        params = vars(self)
        params_str = ', '.join(f"{key}={value}" for key, value in params.items())
        return f"MyDBSCAN class: {params_str}"
    
    def fit_predict(self, X):
        # Проверка, что X является DataFrame
        if not isinstance(X, pd.DataFrame):
            raise ValueError("Input must be a pandas DataFrame")

        # Количество точек в датасете
        n_samples = X.shape[0]

        # Массив для меток кластеров
        labels = -1 * np.ones(n_samples, dtype=int)  # -1 означает шум

        # Массив для пометки точек как посещенных
        visited = np.zeros(n_samples, dtype=bool)

        cluster_id = 0

        for i in range(n_samples):
            if visited[i]:
                continue

            visited[i] = True
            neighbors = self._region_query(X, i)

            # Если соседей меньше, чем min_samples, метим точку как шум
            if len(neighbors) < self.min_samples:
                continue

            # Рекурсивное расширение кластера
            self._expand_cluster(X, labels, i, neighbors, cluster_id, visited)

            cluster_id += 1

        return labels

    def _region_query(self, X, point_idx):
        #Нахождение соседей точки в пределах eps с использованием выбранной метрики
        distances = self._compute_distance(X, point_idx)
        neighbors = np.where(distances <= self.eps)[0]
        return neighbors

    def _compute_distance(self, X, point_idx):
        #Вычисление расстояний в зависимости от выбранной метрики
        if self.metric == 'euclidean':
            return np.linalg.norm(X.values - X.iloc[point_idx].values, axis=1)
        elif self.metric == 'chebyshev':
            return np.max(np.abs(X.values - X.iloc[point_idx].values), axis=1)
        elif self.metric == 'manhattan':
            return np.sum(np.abs(X.values - X.iloc[point_idx].values), axis=1)
        elif self.metric == 'cosine':
            return 1 - cosine_similarity(X.iloc[point_idx:point_idx+1].values, X.values).flatten()
        else:
            raise ValueError(f"Unsupported metric: {self.metric}")

    def _expand_cluster(self, X, labels, point_idx, neighbors, cluster_id, visited):
        #Рекурсивное добавление точек в кластер
        labels[point_idx] = cluster_id

        i = 0
        while i < len(neighbors):
            current_idx = neighbors[i]
            
            if not visited[current_idx]:
                visited[current_idx] = True
                new_neighbors = self._region_query(X, current_idx)

                # Если точка имеет достаточно соседей, добавляем её в кластер
                if len(new_neighbors) >= self.min_samples:
                    neighbors = np.concatenate((neighbors, new_neighbors))

            if labels[current_idx] == -1:  # Если точка еще не помечена, помечаем её
                labels[current_idx] = cluster_id

            i += 1

In [11]:
#Тест метрик
X, y = make_blobs(n_samples=20, centers=3, random_state=42)

# Преобразуем в DataFrame для удобства
X_df = pd.DataFrame(X, columns=['Feature 1', 'Feature 2'])

# Пример 1: Используем Евклидово расстояние
dbscan_euclidean = MyDBSCAN(eps=1.5, min_samples=5, metric='euclidean')
labels_euclidean = dbscan_euclidean.fit_predict(X_df)
print(f"Метки кластеров (Евклидово): {labels_euclidean}")

# Пример 2: Используем Манхэттенское расстояние
dbscan_manhattan = MyDBSCAN(eps=1.5, min_samples=5, metric='manhattan')
labels_manhattan = dbscan_manhattan.fit_predict(X_df)
print(f"Метки кластеров (Манхэттенское): {labels_manhattan}")

# Пример 3: Используем Косинусное расстояние
dbscan_cosine = MyDBSCAN(eps=1.5, min_samples=5, metric='cosine')
labels_cosine = dbscan_cosine.fit_predict(X_df)
print(f"Метки кластеров (Косинусное): {labels_cosine}")

# Пример 4: Используем Расстояние Чебышёва
dbscan_chebyshev = MyDBSCAN(eps=1.5, min_samples=5, metric='chebyshev')
labels_chebyshev = dbscan_chebyshev.fit_predict(X_df)
print(f"Метки кластеров (Чебышёв): {labels_chebyshev}")

Метки кластеров (Евклидово): [ 0  0  0 -1  1  2  0  1  1  1 -1 -1  2  0  2 -1  2  2  1 -1]
Метки кластеров (Манхэттенское): [ 0  0  0 -1 -1 -1  0 -1 -1 -1 -1 -1 -1  0 -1 -1 -1 -1 -1 -1]
Метки кластеров (Косинусное): [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Метки кластеров (Чебышёв): [ 0  0  0  0  1  2  0  1  1  1 -1  1  2  0  2 -1  2  2  1  1]


In [7]:
# Тест обучения-предсказания
X, y = make_blobs(n_samples=20, centers=3, random_state=42)

# Преобразуем в DataFrame для удобства
X_df = pd.DataFrame(X, columns=['Feature 1', 'Feature 2'])

# Создаем экземпляр класса DBSCAN
dbscan = MyDBSCAN(eps=1.5, min_samples=5)

# Выполняем кластеризацию
labels = dbscan.fit_predict(X_df)

# Печатаем метки кластеров для каждой точки
print(labels)

[ 0  0  0 -1  1  2  0  1  1  1 -1 -1  2  0  2 -1  2  2  1 -1]


In [3]:
# Тест класса
model1 = MyDBSCAN()
print(model1)

model2 = MyDBSCAN(eps=5, min_samples=4)
print(model2)

model3 = MyDBSCAN(eps=4)
print(model3)

MyKMeans class: eps=3, min_samples=3
MyKMeans class: eps=5, min_samples=4
MyKMeans class: eps=4, min_samples=3
